# BI output results and speed comparaison
## 1. Continuous variable
### 1.1. Speed comparaison

In [1]:
import os
import sys
newPath = os.path.dirname(os.path.abspath(""))
if newPath not in sys.path:
    sys.path.append(newPath)
#from src.main import*
import time as tm
from src.main_jax import*

2024-04-01 09:58:42.058031: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 09:58:42.058068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 09:58:42.090545: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-01 09:58:42.824308: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
## Model m4.3
d = pd.read_csv('../data/Howell1.csv', sep=';')
d = d[d.age > 18]
#self.df["weight.per.g"].pipe(lambda x: (x - x.mean()) / x.std())
d.weight = d.weight - d.weight.mean()
d.age = d.age - d.age.mean()
formula = dict(main1 = 'height ~ Normal(mu,sigma)',
            likelihood = 'mu ~ alpha + beta * weight',
            prior1 = 'sigma ~ Uniform(0,50)',
            prior2 = 'alpha ~ Normal(178,20)',
            prior3 = 'beta ~ Normal(0,1)')    

start = tm.time()
m4_3 = model(formula, df = d, float = 32)
m4_3.fit(observed_data = dict(height = d.height.astype('float32').values))   
end = tm.time()        
print(f"BI took: {end - start:.4f} seconds")

jax.local_device_count  32
BI took: 2.2279 seconds


/home/sosa/.local/lib/python3.10/site-packages/arviz/data/base.py:221: UserWarning: More chains (2000) than draws (4). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


In [2]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
nest_asyncio.apply()
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass
stan_code = """
data{
  vector[346] height;
  vector[346] weight;
}
parameters{
  real a;
  real<lower=0> b;
  real<lower=0,upper=50> sigma;
}
model{
  vector[346] mu;
  sigma ~ uniform( 0 , 50 );
  b ~ lognormal( 0 , 1 );
  a ~ normal( 178 , 20 );
  for ( i in 1:346 ) {
    mu[i] = a + b* weight[i] ;
  }
  height ~ normal( mu , sigma );  
}
"""
data = {
  'height': d.height.values,
  'weight': d.weight.values,
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)  
df = fit.to_frame()
end = tm.time()  
print(f"Pystan took: {end - start:.4f} seconds")

NameError: name 'd' is not defined

### 1.2. Output comparaison 

In [14]:
pd.DataFrame(
    {
        "tfp": m4_3.summary(round_to = 'none')['mean'],
        "pystan": [df.sigma.mean(),  df.b.mean(), df.a.mean()]
    })

,tfp,pystan
sigma[0],5.141531,5.144897
beta[0],0.905336,0.904733
alpha[0],154.653732,154.649662


## 2. Categorical variable
### 2.1. Speed comparaisons

In [22]:
m5_9 = model()
m5_9.import_csv('../data/milk.csv', sep = ';')
m5_9.df["K"] = m5_9.df["kcal.per.g"].pipe(lambda x: (x - x.mean()) / x.std())
m5_9.index(cols = "clade")
formula = dict(main = 'K ~ Normal(mu,sigma)',
            likelihood = 'mu ~ alpha[index_clade]',
            prior1 = 'alpha~ Normal(0,0.5)',
            prior2 = 'sigma ~ Exponential(1)') 

m5_9.f = formula

start = tm.time()
m5_9.build_model()
m5_9.fit(observed_data = dict(K = m5_9.df.K.astype('float32').values))
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

jax.local_device_count  32
BI took: 1.5272 seconds


/home/sosa/.local/lib/python3.10/site-packages/arviz/data/base.py:221: UserWarning: More chains (2000) than draws (4). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


In [16]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()
stan_code = """
data{
    vector[29] K;
    array[29] int clade_id;
}
parameters{
    vector[4] a;
    real<lower=0> sigma;
}
model{
    vector[29] mu;
    sigma ~ exponential( 1 );
    a ~ normal( 0 , 0.5 );
    for ( i in 1:29 ) {
        mu[i] = a[clade_id[i]];
    }
    K ~ normal( mu , sigma );
    
}
"""
data = {
  'clade_id': m5_9.df.index_clade.values+1,
  'K': m5_9.df.K.values,
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.8s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Excepti

Pystan took: 12.1463 seconds


### 2.2. Output comparaison

In [23]:
pd.DataFrame(
    {
        "tfp": m5_9.summary(round_to = 'none')['mean'],
        "pystan": [df.sigma.mean(),  df['a.1'].mean(), df['a.2'].mean(), df['a.3'].mean(), df['a.4'].mean()]
    })

,tfp,pystan
sigma[0],0.799599,0.801640
alpha[0],-0.461480,-0.459873
alpha[1],0.351941,0.353296
alpha[2],0.636618,0.636981
alpha[3],-0.549415,-0.547616


## 3. Continuous interactions terms (model 8.3)
### 3.1. Speed comparaisons

In [25]:
d = pd.read_csv('../data/tulips.csv', sep = ';')
d["blooms_std"] = d.blooms / d.blooms.max()
d["water_cent"] = d.water - d.water.mean()
d["shade_cent"] = d.shade - d.shade.mean()

formula = dict(
            main = 'blooms_std ~ Normal( mu , sigma ) ',
            likelihood ='mu ~ a + bw*water_cent + bs*shade_cent + bws*water_cent*shade_cent' ,
            prior1 = 'a ~ Normal( 0.5 , 0.25 ) ',
            prior2 = 'bw ~ Normal( 0 , 0.25 ) ',
            prior3 = 'bs ~ Normal( 0 , 0.25 ) ',
            prior4 = 'bws ~ Normal( 0 , 0.25 ) ',
            prior5 = 'sigma ~ Exponential( 1 )',
            )
start = tm.time()
m8_5 = model(formula, d)
m8_5.fit(observed_data = dict(blooms_std =d.blooms_std.astype('float32').values))
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

jax.local_device_count  32
BI took: 1.5952 seconds


/home/sosa/.local/lib/python3.10/site-packages/arviz/data/base.py:221: UserWarning: More chains (2000) than draws (4). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


In [26]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()

stan_code = """
data{
    vector[27] blooms_std;
    array[27] int shade_cent;
    array[27] int water_cent;
}
parameters{
    real a;
    real bw;
    real bs;
    real bws;
    real<lower=0> sigma;
}
model{
    vector[27] mu;
    sigma ~ exponential( 1 );
    bws ~ normal( 0 , 0.25 );
    bs ~ normal( 0 , 0.25 );
    bw ~ normal( 0 , 0.25 );
    a ~ normal( 0.5 , 0.25 );
    for ( i in 1:27 ) {
        mu[i] = a + bw * water_cent[i] + bs * shade_cent[i] + bws * water_cent[i] * shade_cent[i];
    }

    
    blooms_std ~ normal( mu , sigma );
}
"""

data = {
    'blooms_std' : d["blooms_std"].values,
    "water_cent": d["water_cent"].values.astype(int),
    "shade_cent": d["shade_cent"].values.astype(int),
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.7s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 3.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.31 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.32 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.28 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.31 seconds.
  Adjust your expectations accordingly!


Pystan took: 12.1231 seconds


### 3.2. Output comparaison

In [27]:
pd.DataFrame(
    {
        "tfp": m8_5.summary(round_to='none')['mean'],
        "pystan": [df.sigma.mean(),  df['bws'].mean(), df['bw'].mean(), df['bs'].mean(), df['a'].mean()]
    })

,tfp,pystan
sigma[0],0.142805,0.142327
bws[0],-0.142726,-0.143176
bw[0],0.205038,0.206099
bs[0],-0.112257,-0.112393
a[0],0.358838,0.358277


## 4. Binomial model
### 4.1. Speed comparaisons

In [29]:
d = pd.read_csv('../data/chimpanzees.csv', sep = ';')
d["treatment"] = 1 + d.prosoc_left + 2 * d.condition
d["side"] = d.prosoc_left  # right 0, left 1
d["cond"] = d.condition  # no partner 0, partner 1

d_aggregated = (
    d.groupby(["treatment", "actor", "side", "cond"])["pulled_left"].sum().reset_index()
)
d_aggregated.rename(columns={"pulled_left": "left_pulls"}, inplace=True)
d_aggregated["actor_id"] = d_aggregated["actor"].values - 1

formula = dict(
    main = 'pulled_left ~ Binomial( 1 , logits = p )' ,
    likelihood = 'p ~ a' ,
    prior1 = 'a ~ Normal( 0 , 10 )'
)
start = tm.time()
m11_1 = model(formula, d)
m11_1.fit(observed_data = dict(pulled_left =d.pulled_left.astype('float32').values))
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

jax.local_device_count  32
BI took: 1.4346 seconds


/home/sosa/.local/lib/python3.10/site-packages/arviz/data/base.py:221: UserWarning: More chains (2000) than draws (4). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


In [30]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()

stan_code = """
data{
    array[504] int pulled_left;
}
parameters{
    real a;
}
model{
    real p;
    a ~ normal( 0 , 10 );
    p = a;
    p = inv_logit(p);
    pulled_left ~ binomial( 1 , p );
    
}
"""

data = {
    'pulled_left' : d["pulled_left"].values.astype(int)
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 10.2s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!


Pystan took: 10.5593 seconds


### 4.2. Output comparaison

In [31]:
pd.DataFrame(
    {
        "tfp": m11_1.summary(round_to='none')['mean'],
        "pystan": [df.a.mean()]
    })

,tfp,pystan
a[0],0.323161,0.321119


##  5. Binomial with indices
### 5.1.Speed comparaisons

In [32]:
from src.main import*
d = pd.read_csv('../data/chimpanzees.csv', sep = ';')
d.actor = d.actor - 1
d["treatment"] = d.prosoc_left + 2 * d.condition
d[["actor", "prosoc_left", "condition", "treatment"]]

formula = dict(
    main = 'pulled_left ~ Binomial(1 , p )' ,
    likelihood = 'p ~ a[actor] + b[treatment]' ,
    prior1 = 'a ~ Normal(0, 1.5)',
    prior2 = 'b ~ Normal(0, 0.5)'
)

start = tm.time()
m11_4 = model(formula, d, float = 32)
m11_4.fit(observed_data = dict(pulled_left =d.pulled_left.astype('float32').values))
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")


Tensor("JointDistributionNamedAutoBatched/log_prob/add_2:0", shape=(1,), dtype=float32)
BI took: 5.1156 seconds


In [33]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()

stan_code = """
data{
    array[504] int pulled_left;
    array[504] int treatment;
    array[504] int actor;
}
parameters{
    vector[7] a;
    vector[4] b;
}
model{
    vector[504] p;
    b ~ normal( 0 , 0.5 );
    a ~ normal( 0 , 1.5 );
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + b[treatment[i]];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
generated quantities{
    vector[504] log_lik;
    vector[504] p;
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + b[treatment[i]];
        p[i] = inv_logit(p[i]);
    }
    for ( i in 1:504 ) log_lik[i] = binomial_lpmf( pulled_left[i] | 1 , p[i] );
    
}
"""

data = {
    'pulled_left' : d["pulled_left"].values.astype(int),
    'treatment' : d["treatment"].values.astype(int) +1,
    'actor' : d["actor"].values.astype(int) +1
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.8s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling:  79% (7900/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 5.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.56 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.38 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.39 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.61 seconds.
  Adjust your expectations accordingly!


Pystan took: 15.5431 seconds


### 5.2. Output comparaison

In [34]:
pd.DataFrame(
    {
        "tfp": m11_4.summary(round_to='none')['mean'],
        "pystan": [df['b.1'].mean(), df['b.2'].mean(), df['b.3'].mean(), df['b.4'].mean(),
                   df['a.1'].mean(), df['a.2'].mean(), df['a.3'].mean(), df['a.4'].mean(), df['a.5'].mean(), df['a.6'].mean(), df['a.7'].mean()]
    })

,tfp,pystan
b[0],-0.041349,-0.031648
b[1],0.480509,0.490967
b[2],-0.356620,-0.375165
b[3],0.375912,0.376894
a[0],-0.468963,-0.455304
a[1],3.874392,3.887521
a[2],-0.752882,-0.757069
a[3],-0.749913,-0.758169
a[4],-0.468166,-0.459580
a[5],0.461863,0.471230


## 6. Poisson
### 6.1. Speed comparaison

In [35]:
from src.main import*
d = pd.read_csv('../data/Kline.csv', sep = ';')
d["P"] = d.population.pipe(np.log).pipe(lambda x: (x - x.mean()) / x.std())
d["cid"] = (d.contact == "high").astype(int)
d['pLog'] = tf.math.log(d.P).numpy()
formula = dict(main = 'total_tools ~ Poisson(log_rate = lambda)',
               likelihood = 'lambda ~ alpha[cid] + beta[cid]*P',
               prior1 = 'alpha ~ Normal(3,0.5)',
               prior2 = 'beta ~ Normal(0,0.2)')
start = tm.time()
m11_10 = model(formula, d)
m11_10.fit(observed_data = dict(total_tools =d.total_tools.astype('float32').values))
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

Tensor("JointDistributionNamedAutoBatched/log_prob/add_2:0", shape=(1,), dtype=float32)
BI took: 4.6995 seconds


In [36]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()

stan_code = """ 
data{
    array[10] int T;
    vector[10] P;
    array[10] int cid;
}
parameters{
    vector[2] a;
    vector[2] b;
}
model{
    vector[10] lambda;
    b ~ normal( 0 , 0.2 );
    a ~ normal( 3 , 0.5 );
    for ( i in 1:10 ) {
       lambda[i] = a[cid[i]] + b[cid[i]] * P[i];
       lambda[i] = exp(lambda[i]);
    }
    T ~ poisson( lambda );
}
generated quantities{
    vector[10] log_lik;
    vector[10] lambda;
    for ( i in 1:10 ) {
        lambda[i] = a[cid[i]] + b[cid[i]] * P[i];
        lambda[i] = exp(lambda[i]);
    }
    for ( i in 1:10 ) log_lik[i] = poisson_lpmf( T[i] | lambda[i]);
    
}
"""

data = {
    'T' : d["total_tools"].values.astype(int),
    'P' : d["P"].values.astype(float),
    'cid' : d["cid"].values.astype(int) +1
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 13.1s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 8e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.08 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 8e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.08 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 9e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.09 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.1 seconds.
  Adjust your expectations accordingly!


Pystan took: 13.4400 seconds


### 6.2. Output comparaison 

In [37]:
pd.DataFrame(
    {
        "tfp": m11_10.summary(round_to='none')['mean'],
        "pystan": [df['b.1'].mean(), df['b.2'].mean(), 
                   df['a.1'].mean(), df['a.2'].mean()]
    })

,tfp,pystan
beta[0],0.377608,0.377975
beta[1],0.186534,0.190583
alpha[0],3.318159,3.319431
alpha[1],3.615976,3.610163


## 7. Negative binomial (PB estimation)
### 7.1. Speed comparaison

In [38]:
num_days = 30
y = tfd.Poisson(rate=1.5).sample((num_days,))
num_weeks = 4
y_new = tfd.Poisson(rate=0.5 * 7).sample((num_weeks,))
y_all = np.concatenate([y, y_new])
exposure = np.concatenate([np.repeat(1, 30), np.repeat(7, 4)])
monastery = np.concatenate([np.repeat(0, 30), np.repeat(1, 4)])
d = pd.DataFrame.from_dict(dict(y=y_all, days=exposure, monastery=monastery))
d["log_days"] = d.days.pipe(np.log)

formula = dict(main = 'y ~ Poisson(rate = lambda)',
               likelihood = 'lambda ~ tf.exp(log_days + alpha +  beta * monastery)',
               prior1 = 'alpha ~ Normal(0,1)',
               prior2 = 'beta ~ Normal(0,1)')
start = tm.time()
m11_12 = model(formula, d, float=32)
m11_12.fit(observed_data = dict(y =d.y.astype('float32').values))
end = tm.time()
print(f"BI took: {end - start:.4f} seconds")

Tensor("JointDistributionNamedAutoBatched/log_prob/add_2:0", shape=(1,), dtype=float32)
BI took: 4.6588 seconds


In [41]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()

stan_code =""" 
data{
    array[34] int days;
    array[34] int y;
    array[34] int monastery;
    vector[34] log_days;
}
parameters{
    real a;
    real b;
}
model{
    vector[34] lambda;
    b ~ normal( 0 , 1 );
    a ~ normal( 0 , 1 );
    for ( i in 1:34 ) {
        lambda[i] = log_days[i] + a + b * monastery[i];
        lambda[i] = exp(lambda[i]);
    }
    
    y ~ poisson( lambda );
    
}
"""

data = {
    'days' : d["days"].values.astype(int),
    'y' : d["y"].values.astype(int),
    'monastery' : d["monastery"].values.astype(int) +1,
    'log_days' : d["log_days"].values.astype(float),
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.0s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 7e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.07 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 8e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.08 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 8e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.08 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 8e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.08 seconds.
  Adjust your expectations accordingly!


Pystan took: 11.4282 seconds


### Output comparaison (PB estimation)

In [42]:
pd.DataFrame(
    {
        "tfp": m11_12.summary(round_to='none')['mean'],
        "pystan": [df['b'].mean(), df['a'].mean()]
    })

,tfp,pystan
beta[0],-0.979285,-0.866921
alpha[0],0.389058,1.210285


## 8. Multinomial
TODO : This model can't account for indices in the formula nor varying intercepts of varying effects.
### 8.1. Speed comparaison

In [43]:
# simulate career choices among 500 individuals
N = 500  # number of individuals
income = np.array([1, 2, 5])  # expected income of each career
score = 0.5 * income  # scores for each career, based on income

# next line converts scores to probabilities
p = tf.nn.softmax(score)

# now simulate choice
# outcome career holds event type values, not counts
career = np.repeat(np.nan, N)  # empty vector of choices for each individual

# sample chosen career for each individual
for i in range(N):
    career[i] = tfd.Categorical(probs=p).sample()

career = career.astype(int)
result = [income[index] for index in career]
data = {'career': career, 'income': result}
d = pd.DataFrame(data)

In [ ]:
formula = dict(main = 'y ~ Poisson(log_rate = p)',
               likelihood = 'p ~ alpha[income] + beta * income',
               prior1 = 'alpha ~ Normal(0,1)',
               prior2 = 'beta ~ Normal(0, 0.5)')
m11_13 = model(formula, d)


2024-03-15 14:32:43.869962: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:32:43.870024: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:32:43.870038: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:32:43.870218: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:32:43.870231: I tensorflow/core/common_runtime/gpu/gpu

{'alpha': "tfd.Sample(tfd.Normal(0,1, name = 'prior1'), sample_shape = 3)",
 'beta': "tfd.Sample(tfd.Normal(0,0.5, name = 'prior2'), sample_shape = 1)"}

In [44]:
formula = dict(main = 'y ~ Categorical(probs = p, cat = income)',
               likelihood = 'p ~ alpha[income] + beta * income',
               prior1 = 'alpha ~ Normal(0, 1)',
               prior2 = 'beta ~ HalfNormal(0.5)'
               )
start = tm.time()   
m11_13 = model(formula, d)
m11_13.fit(observed_data = dict(y =d.career.astype('float32').values), num_chains=4)
end = tm.time()
print(f"BI took: {end - start:.4f} seconds")

tf.nn.softmax(tf.stack([alpha[0]+beta*1.0,alpha[1]+beta*2.0,0 +beta*2.0], axis=1))
Tensor("JointDistributionNamedAutoBatched/log_prob/add_2:0", shape=(1,), dtype=float32)
BI took: 4.6478 seconds


In [47]:
import pymc as pm
with pm.Model() as m11_13_pm:
    a = pm.Normal("a", 0.0, 1.0, shape=2)  # intercepts
    b = pm.HalfNormal("b", 0.5)  # association of income with choice

    s0 = a[0] + b * income[0]
    s1 = a[1] + b * income[1]
    s2 = 0.0 + b * income[2]  # pivoting the intercept for the third category
    s = pm.math.stack([s0, s1, s2])

    p_ = pm.math.softmax(s)
    career_obs = pm.Categorical("career", p=p_, observed=career)

    trace_11_13 = pm.sample(tune=2000, target_accept=0.99)

Task exception was never retrieved
future: <Task finished name='Task-617' coro=<HttpstanClient.post() done, defined at /home/sosa/.local/lib/python3.10/site-packages/stan/common.py:50> exception=ServerDisconnectedError('Server disconnected')>
Traceback (most recent call last):
  File "/home/sosa/.local/lib/python3.10/site-packages/stan/model.py", line 519, in build
    return asyncio.run(go())
  File "/home/sosa/.local/lib/python3.10/site-packages/nest_asyncio.py", line 31, in run
    return loop.run_until_complete(task)
  File "/home/sosa/.local/lib/python3.10/site-packages/nest_asyncio.py", line 93, in run_until_complete
    self._run_once()
  File "/home/sosa/.local/lib/python3.10/site-packages/nest_asyncio.py", line 116, in _run_once
    event_list = self._selector.select(timeout)
  File "/usr/lib/python3.10/selectors.py", line 469, in select
    fd_event_list = self._selector.poll(timeout, max_ev)
KeyboardInterrupt

During handling of the above exception, another exception occurre

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Sampling 4 chains for 2_000 tune and 1_000 draw iterations (8_000 + 4_000 draws total) took 7 seconds.


In [48]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()

stan_code =""" 
data{
    int N; // number of individuals
    int K; // number of possible careers
    array[N] int career; // outcome
    vector[K] career_income;
}
parameters{
    vector[K-1] a; // intercepts
    real<lower=0> b; // association of income with choice
}
model{
    vector[K] p;
    vector[K] s;
    a ~ normal( 0 , 1 );
    b ~ normal( 0 , 0.5 );
    s[1] = a[1] + b*career_income[1];
    s[2] = a[2] + b*career_income[2];
    s[3] = 0; // pivot
    p = softmax( s );
    career ~ categorical( p );
}
"""

data = {
    'N' : 500,
    'K' : 3,
    'career' : d["career"].values.astype(int) + 1,
    'career_income' : d["income"].unique().astype(int).tolist(),
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 12.2s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 1.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.11 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 9e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.09 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.1 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 9e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.09 seconds.
  Adjust your expectations accordingly!


Pystan took: 12.7902 seconds


In [50]:
tmp = az.summary(trace_11_13, round_to=2)['mean']
pd.DataFrame(
    {
        "tfp": m11_13.summary(round_to='none')['mean'],
        "pystan": [df['b'].mean(), df['a.1'].mean(), df['a.2'].mean()],
        "pymc": [tmp[2], tmp[0], tmp[1]]
    })
# It seems that beta should be 0.5

/tmp/ipykernel_212934/3955049630.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "pymc": [tmp[2], tmp[0], tmp[1]]


,tfp,pystan,pymc
beta[0],0.572273,0.052870,0.46
alpha[0],-1.634189,-2.454034,-0.42
alpha[1],-1.395898,-1.498306,-0.05


## 9. Beta binomial (PB estimation)
### 9.1. Speed comparaison

In [51]:
from src.main import*
d = pd.read_csv('../data/UCBadmit.csv', sep = ';')
d["gid"] = (d["applicant.gender"] != "male").astype(int)
len(d.applications)
formula = dict(main = 'y ~ BetaBinomial(applications, concentration1 = pbar*theta, concentration0 = (1 - pbar) * theta)',
               likelihood = 'pbar ~ sigmoid(alpha[gid])',
               likelihood2 = 'theta ~ phi + 2.0',
               prior1 = 'alpha ~ Normal(0.,1.5)',
               prior2 = 'phi ~ Exponential(1)'
               )
start = tm.time()
m12_1 = model(formula, d)
m12_1.fit(observed_data = dict(y =d.admit.astype('float32').values))
end = tm.time()
print(f"BI took: {end - start:.4f} seconds")

Tensor("JointDistributionNamedAutoBatched/log_prob/add_2:0", shape=(1,), dtype=float32)
BI took: 5.6142 seconds


In [52]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()
stan_code =""" 
data{
    array[12] int N;
    array[12] int A;
    array[12] int gid;
}
parameters{
    vector[2] a;
    real<lower=0> phi;
}
transformed parameters{
    real theta;
    theta = phi + 2;
}
model{
    vector[12] pbar;
    phi ~ exponential( 1 );
    a ~ normal( 0 , 1.5 );
    for ( i in 1:12 ) {
        pbar[i] = a[gid[i]];
        pbar[i] = inv_logit(pbar[i]);
    }
    A ~ beta_binomial( N , pbar*theta , (1-pbar)*theta );    
}
"""

data = {
    'A' : d["admit"].values.astype(int),
    'N' : d["applications"].values.astype(int),
    'gid' : d["gid"].values.astype(int) +1,
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 12.4s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 9e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.09 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: beta_binomial_lpmf: Second prior sample size parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_bhvvarns/model_aokd6ccs.stan', line 23, column 4 to column 57)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Gradient evaluation took 8e-06 seconds
  1000 transitions using 10 

Pystan took: 12.8920 seconds


### 9.2. Output comparaison (PB estimation)

In [53]:
pd.DataFrame(
    {
        "tfp": m12_1.summary(round_to='none')['mean'],
        "pystan": [df['phi'].mean(), df['a.1'].mean(), df['a.2'].mean()]
    })

,tfp,pystan
phi[0],-0.820031,1.025307
alpha[0],0.535661,-0.446708
alpha[1],0.320808,-0.321521


## 10. Negative-binomial (WIP)
### 10.1. Speed comparaison

In [ ]:
#from src.main import*
#d = pd.read_csv('../data/Kline.csv', sep = ';')
#formula = dict(main = 'y ~ BetaBinomial(12,  concentration1 = pbar/phi, concentration0 = g_rate)',
#               likelihood1 = 'pbar ~ tf.exp(alpha[gid])*tf.math.pow(beta[gid])/gamma',
#               likelihood2 = 'g_rate ~ 1/phi',
#               prior1 = 'alpha ~ Normal(1,1)',
#               prior2 = 'beta ~ Exponential(1)',
#               prior3 = 'gamma ~ Exponential(1)',
#               prior4 = 'phi ~ Exponential(1)',
#               )
#start = tm.time()
#m12_1 = model(formula, d)

In [ ]:
#import tensorflow as tf
#import tensorflow_probability as tfp
#
#tf.random.set_seed(42)
#
## Define the model
#def model():
#    m = {}
#    m['alpha'] = tfp.distributions.Sample(tfp.distributions.Normal(1, 1), sample_shape=2)
#    m['beta'] = tfp.distributions.Sample(tfp.distributions.Exponential(1), sample_shape=2)
#    m['gamma'] = tfp.distributions.Sample(tfp.distributions.Exponential(1), sample_shape=1)
#    m['phi'] = tfp.distributions.Sample(tfp.distributions.Exponential(1), sample_shape=1)
#    
#    # Define the distribution 'y' which depends on alpha, beta, gamma, and phi
#    def y_dist(phi, alpha, beta, gamma):
#        concentration1 = ((tf.exp(alpha) * beta) / gamma) / phi
#        concentration0 = 1. / phi
#        return tfp.distributions.Independent(
#            tfp.distributions.BetaBinomial(12, concentration1=concentration1, concentration0=concentration0),
#            reinterpreted_batch_ndims=1)
#
#    m['y'] = y_dist
#    return tfp.distributions.JointDistributionNamedAutoBatched(m)
#
## Sample from the model
#sampled_values = model().sample(alpha=tf.random.normal((2,)), beta=tfp.distributions.Exponential(rate=1.).sample((2,)), gamma=tfp.distributions.Exponential(rate=1.).sample((1,)), phi=tfp.distributions.Exponential(rate=1.).sample((1,)))
#
#print(sampled_values)



In [58]:
#df = d
#m = {}
#m['alpha']= tfd.Sample(tfd.Normal(1,1, name = 'prior1'), sample_shape = 2)
#m['beta']= tfd.Sample(tfd.Exponential(1, name = 'prior2'), sample_shape = 2)
#m['gamma']= tfd.Sample(tfd.Exponential(1, name = 'prior3'), sample_shape = 1)
#m['phi']= tfd.Sample(tfd.Exponential(1, name = 'prior4'), sample_shape = 1)
#m['y'] = lambda phi, alpha, beta, gamma : tfd.Independent(
#    tfd.BetaBinomial(12,
#                     concentration1=(
#                         tf.exp(tf.squeeze(tf.gather(alpha,tf.cast(df.gid.astype('float32').values, dtype=tf.int32), axis = -1)))
#                         *( tf.math.powtf.squeeze(tf.gather(beta,tf.cast(df.gid.astype('float32').values, dtype=tf.int32), axis = -1)))
#                         /gamma)/phi,
#                     concentration0=(1/phi), name ='main'), reinterpreted_batch_ndims=1)
#m = tfd.JointDistributionNamedAutoBatched(m)
#m.sample()

## 11. Zero inflated outcomes (PB estimation)
### 11.1. Speed comparaison

In [59]:
import random
random.seed(42)
# Define parameters
prob_drink = 0.2  # 20% of days
rate_work = 1     # average 1 manuscript per day

# sample one year of production
N = 365

np.random.seed(365)
drink = np.random.binomial(1, prob_drink, N)
y = (1 - drink) * np.random.poisson(rate_work, N)
d = pd.DataFrame(y)
formula = dict(main = 'y ~ ZeroInflatedNegativeBinomial(total_count = 365, inflated_loc_logits = p, logits = AL)',
               likelihood = "p ~ ap",
               likelihood2 = "AL ~ tf.math.log(al)",
               prior1 = 'ap ~ Normal(-1.5 , 1)',
               prior2 = 'al ~ Normal(1,0.5)'
               )

start = tm.time()
m12_3 = model()
m12_3 = model(formula)       
m12_3.fit(observed_data = dict(y = d.iloc[:,0].astype('float32').values))
end = tm.time()
print(f"BI took: {end - start:.4f} seconds")

Tensor("JointDistributionNamedAutoBatched/log_prob/add_2:0", shape=(1,), dtype=float32)
BI took: 4.9368 seconds


In [60]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()
stan_code = """ 
data{
    array[365] int y;
}
parameters{
    real ap;
    real al;
}
model{
    real p;
    real lambda;
    al ~ normal( 1 , 0.5 );
    ap ~ normal( -1.5 , 1 );
    lambda = al;
    lambda = exp(lambda);
    p = ap;
    p = inv_logit(p);
    for ( i in 1:365 ) {
        if ( y[i]==0 )
            target += log_mix( p , 0 , poisson_lpmf(0|lambda) );
        if ( y[i] > 0 )
            target += log1m( p ) + poisson_lpmf(y[i] | lambda );
    }
}
"""
data = {
    'y' :d.iloc[:,0].values.astype(int)
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 10.2s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 9.5e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.95 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 7.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.71 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 7.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.71 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 9.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.92 seconds.
  Adjust your expectations accordingly!


Pystan took: 11.5214 seconds


### 11.2. Output comparaison (PB estimation)

In [61]:
pd.DataFrame(
    {
        "tfp": m12_3.summary(round_to='none')['mean'],
        "pystan": [df['ap'].mean(), df['al'].mean()]
    })

,tfp,pystan
ap[0],-1.965017,-1.356490
al[0],0.001720,0.109806


## 12. Varying interceps
### 12.1. Speed comparaison

In [5]:
d = pd.read_csv('../data/reedfrogs.csv', sep = ';')
d["tank"] = np.arange(d.shape[0])
formula = dict(main = 'y ~ Binomial(total_count = density, logits = p)',
               likelihood = 'p ~ alpha[tank]', 
               prior = 'alpha ~ Normal(a_bar, sigma)',
               prior1 = 'a_bar ~ Normal(0.,1.5)',
               prior2 = 'sigma ~ Exponential(1)'
               )

start = tm.time()   
m13_2 = model(formula, d, float=32)
m13_2.fit(observed_data = dict(y =d.surv.astype('float32').values), num_chains= 4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

jax.local_device_count  32
BI took: 4.8834 seconds


/home/sosa/.local/lib/python3.10/site-packages/arviz/data/base.py:221: UserWarning: More chains (2000) than draws (4). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


In [123]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache


nest_asyncio.apply()
stan_code = """ 
data{
    array[48] int N;
    array[48] int S;
    array[48] int tank;
}
parameters{
    vector[48] a;
    real a_bar;
    real<lower=0> sigma;
}
model{
    vector[48] p;
    sigma ~ exponential( 1 );
    a_bar ~ normal( 0 , 1.5 );
    a ~ normal( a_bar , sigma );
    for ( i in 1:48 ) {
        p[i] = a[tank[i]];
        p[i] = inv_logit(p[i]);
    }
    S ~ binomial( N , p );
}
generated quantities{
    vector[48] log_lik;
    vector[48] p;
    for ( i in 1:48 ) {
        p[i] = a[tank[i]];
        p[i] = inv_logit(p[i]);
    }
    for ( i in 1:48 ) log_lik[i] = binomial_lpmf( S[i] | N[i] , p[i] );    
}
"""
data = {
    'S' : d['surv'].values.astype(int),
    'N' : d['density'].values.astype(int),
    'tank' : d['tank'].values.astype(int)+1,
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: found in cache, done.Sampling:   0%
Sampling:   0% (1/10000)
Sampling:  25% (2501/10000)
Sampling:  50% (5001/10000)
Sampling:  75% (7501/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 0.002889 seconds
  1000 transitions using 10 leapfrog steps per transition would take 28.89 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.00289 seconds
  1000 transitions using 10 leapfrog steps per transition would take 28.9 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in '/tmp/httpstan_ixx0of3a/model_nkw3miai.stan', line 16, column 4 to column 32)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if

Pystan took: 1.9976 seconds


### 12.3 Output comparaison

In [76]:
pd.DataFrame(
    {
        "tfp": m13_2.summary(round_to='none')['mean'],
        "pystan": [df['sigma'].mean(), df['a_bar'].mean(),
                   df['a.1'].mean(), df['a.2'].mean(),
                   df['a.3'].mean(), df['a.4'].mean(),
                   df['a.5'].mean(), df['a.6'].mean(),
                   df['a.7'].mean(), df['a.8'].mean(),
                   df['a.9'].mean(), df['a.10'].mean(),
                   df['a.11'].mean(), df['a.12'].mean(),
                   df['a.13'].mean(), df['a.14'].mean(),
                   df['a.15'].mean(), df['a.16'].mean(),
                   df['a.17'].mean(), df['a.18'].mean(),
                   df['a.19'].mean(), df['a.20'].mean(),
                   df['a.21'].mean(), df['a.22'].mean(),
                   df['a.23'].mean(), df['a.24'].mean(),
                   df['a.25'].mean(), df['a.26'].mean(),
                   df['a.27'].mean(), df['a.28'].mean(),
                   df['a.29'].mean(), df['a.30'].mean(),
                   df['a.31'].mean(), df['a.32'].mean(),
                   df['a.33'].mean(), df['a.34'].mean(),
                   df['a.35'].mean(), df['a.36'].mean(),
                   df['a.37'].mean(), df['a.38'].mean(),
                   df['a.39'].mean(), df['a.40'].mean(),
                   df['a.41'].mean(), df['a.42'].mean(),
                   df['a.43'].mean(), df['a.44'].mean(),
                   df['a.45'].mean(), df['a.46'].mean(),
                   df['a.47'].mean(), df['a.48'].mean()
                   ]
    })

,tfp,pystan
sigma[0],1.626218,1.620453
a_bar[0],1.351135,1.348769
"alpha[0, 0]",2.148302,2.140860
"alpha[0, 1]",3.182271,3.077484
"alpha[0, 2]",1.007853,0.997670
"alpha[0, 3]",3.174410,3.075111
"alpha[0, 4]",2.165270,2.146803
"alpha[0, 5]",2.177249,2.135257
"alpha[0, 6]",3.050784,3.081528
"alpha[0, 7]",2.126648,2.135861


## 13. Varying effects (Building issue)
### 13.1. Speed comparaison

In [2]:
from src.main import*
from tensorflow_probability import bijectors as tfb
a = 3.5  # average morning wait time
b = -1  # average difference afternoon wait time
sigma_a = 1  # std dev in intercepts
sigma_b = 0.5  # std dev in slopes
rho = -0.7  # correlation between intercepts and slopes
Mu = tf.constant([a, b])
cov_ab = sigma_a * sigma_b * rho
Sigma = tf.constant([[sigma_a ** 2, cov_ab], [cov_ab, sigma_b ** 2]])
tf.transpose(tf.reshape(tf.constant([1, 2, 3, 4]), (2, 2)))
sigmas = tf.constant([sigma_a, sigma_b])  # standard deviations
Rho = tf.constant([[1, rho], [rho, 1]])  # correlation matrix

# now matrix multiply to get covariance matrix
Sigma = tf.linalg.tensor_diag(sigmas) @ Rho @ tf.linalg.tensor_diag(sigmas)
Sigma
N_cafes = 20

def build_vary_effects():
    _seed = 5
    tf.random.set_seed(_seed)

    seed = tfp.util.SeedStream(_seed, salt="vary_effects")

    Mu = tf.constant([a, b])

    vary_effects = tfd.MultivariateNormalTriL(
        loc=Mu, scale_tril=tf.linalg.cholesky(Sigma)
    ).sample((N_cafes,), seed=seed())

    return vary_effects

vary_effects = build_vary_effects()
a_cafe = vary_effects[:, 0]
b_cafe = vary_effects[:, 1]
N_visits = 10
afternoon = np.tile(np.arange(2), N_visits * N_cafes // 2)
cafe_id = np.repeat(np.arange(N_cafes), N_visits)

def generate_data_frame():
    sigma = 0.5  # std dev within cafes

    _seed = 22
    tf.random.set_seed(_seed)

    seed = tfp.util.SeedStream(_seed, salt="generate_data_frame")

    mu = tf.gather(a_cafe, cafe_id) + tf.gather(b_cafe, cafe_id) * afternoon

    wait = tfd.Normal(loc=mu, scale=sigma).sample(seed=seed())
    d = pd.DataFrame(dict(cafe=cafe_id, afternoon=afternoon, wait=wait))

    return d
d = generate_data_frame()

In [3]:
from src.main_jax import*
formula = dict(main = 'y ~ Normal(mu, sigma)',
likelihood1 = 'mu ~ a_cafe_b_cafe[cafe] + a_cafe_b_cafe[cafe]*afternoon',
prior0 = 'a_cafe_b_cafe ~ MultivariateNormalTriL(concat([alpha, beta],axis=-1), LinearOperatorDiag(sigma_alpha_beta, Rho))',
prior1 = 'sigma ~ Exponential(1)',
prior2 = 'sigma_alpha_beta ~ Exponential(1)',
prior3 = 'alpha ~ Normal(5,2)',
prior4 = 'beta ~ Normal(-1,0.5)',
prior5 = 'Rho ~ LKJ(2,2)',
)
start = tm.time()
m14_1 = model(formula, d)
#m14_1.fit(observed_data = dict(y = d.wait.astype('float32').values),num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


jax.local_device_count  32
BI took: 0.1131 seconds


In [4]:
m14_1.main_dict

{'y': "lambda sigma, a_cafe_b_cafe : tfd.Independent(tfd.Normal( jnp.squeeze(jnp.take(jnp.take(a_cafe_b_cafe, 0, axis=-1),jnp.array(jnp.array(df.cafe.values, dtype=jnp.float32), dtype=jnp.int32), axis = -1))+ jnp.squeeze(jnp.take(jnp.take(a_cafe_b_cafe, 1, axis=-1),jnp.array(jnp.array(df.cafe.values, dtype=jnp.float32), dtype=jnp.int32), axis = -1))*jnp.array(df.afternoon.values, dtype=jnp.float32),sigma, name ='main'), reinterpreted_batch_ndims=1)"}

In [5]:
m14_1.prior_dict

{'a_cafe_b_cafe': 'lambda sigma_alpha_beta, alpha, beta, Rho: tfd.Sample(tfd.MultivariateNormalTriL(jnp.concat([alpha,beta],axis=-1),jax_LinearOperatorDiag(sigma_alpha_beta,Rho)), sample_shape = 20)',
 'sigma': "tfd.Sample(tfd.Exponential(1, name = 'prior1'), sample_shape = 1)",
 'sigma_alpha_beta': "tfd.Sample(tfd.Exponential(1, name = 'prior2'), sample_shape = 2)",
 'alpha': "tfd.Sample(tfd.Normal(5,2, name = 'prior3'), sample_shape = 1)",
 'beta': "tfd.Sample(tfd.Normal(-1,0.5, name = 'prior4'), sample_shape = 1)",
 'Rho': "tfd.Sample(tfd.LKJ(2,2, name = 'prior5'), sample_shape = ())"}

In [15]:
@jit
def jax_LinearOperatorDiag2(s, cov):    
    def multiply_with_s(a):
        return jnp.multiply(a, s)
    vectorized_multiply = vmap(multiply_with_s)
    return jnp.transpose(vectorized_multiply(cov))

In [27]:
sigma= tfd.Exponential(1, name = 'prior1').sample(seed = jnp.array(init_key), sample_shape = 1)
sigma_alpha_beta = tfd.Exponential(1, name = 'prior2').sample(seed = jnp.array(init_key), sample_shape = 2)
alpha = tfd.Normal(5,2, name = 'prior3').sample(seed = jnp.array(init_key), sample_shape = 1)
beta = tfd.Normal(-1,0.5, name = 'prior4').sample(seed = jnp.array(init_key), sample_shape = 1)
Rho = tfd.LKJ(int(2),int(2), name = 'prior5').sample(seed = jnp.array(init_key), sample_shape = ())
a_cafe_b_cafe=  tfd.MultivariateNormalTriL(jnp.concat([alpha,beta],axis=-1), jax_LinearOperatorDiag2(sigma_alpha_beta,Rho)).sample(seed = jnp.array(init_key), sample_shape =  20)
jnp.squeeze(jnp.take(jnp.take(a_cafe_b_cafe, 0, axis=-1),jnp.array(jnp.array(d.cafe.values, dtype=jnp.float32), dtype=jnp.int32), axis = -1))+ jnp.squeeze(jnp.take(jnp.take(a_cafe_b_cafe, 1, axis=-1),jnp.array(jnp.array(d.cafe.values, dtype=jnp.float32), dtype=jnp.int32), axis = -1))*jnp.array(d.afternoon.values, dtype=jnp.float32)

/home/sosa/.local/lib/python3.10/site-packages/tensorflow_probability/python/internal/backend/jax/random_generators.py:283: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in zeros is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  minval = minval + np.zeros([1] * final_rank, dtype=dtype)
/home/sosa/.local/lib/python3.10/site-packages/tensorflow_probability/python/internal/backend/jax/random_generators.py:284: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'>  is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return jaxrand.randint(key=seed, shape=shape, minval=minval, maxval=maxv

Array([3.4726374, 2.3088014, 3.4726374, 2.3088014, 3.4726374, 2.3088014,
       3.4726374, 2.3088014, 3.4726374, 2.3088014, 3.4590604, 2.1139083,
       3.4590604, 2.1139083, 3.4590604, 2.1139083, 3.4590604, 2.1139083,
       3.4590604, 2.1139083, 3.4707913, 1.8273127, 3.4707913, 1.8273127,
       3.4707913, 1.8273127, 3.4707913, 1.8273127, 3.4707913, 1.8273127,
       3.4684947, 2.1962986, 3.4684947, 2.1962986, 3.4684947, 2.1962986,
       3.4684947, 2.1962986, 3.4684947, 2.1962986, 3.4703252, 2.3336573,
       3.4703252, 2.3336573, 3.4703252, 2.3336573, 3.4703252, 2.3336573,
       3.4703252, 2.3336573, 3.4624426, 1.7659965, 3.4624426, 1.7659965,
       3.4624426, 1.7659965, 3.4624426, 1.7659965, 3.4624426, 1.7659965,
       3.476229 , 2.5140269, 3.476229 , 2.5140269, 3.476229 , 2.5140269,
       3.476229 , 2.5140269, 3.476229 , 2.5140269, 3.4690316, 2.07941  ,
       3.4690316, 2.07941  , 3.4690316, 2.07941  , 3.4690316, 2.07941  ,
       3.4690316, 2.07941  , 3.4648366, 2.166154 , 

In [16]:
jnp.concat([alpha,beta],axis=-1)

Array([ 3.4668336, -1.3832916], dtype=float32)

In [22]:
tfd.MultivariateNormalTriL(jnp.concat([alpha,beta],axis=-1), jax_LinearOperatorDiag2(sigma_alpha_beta,Rho)).sample(seed = jnp.array(init_key), sample_shape =  20)

ValueError: Dimensions 1 and 2 are not compatible

In [17]:
jax_LinearOperatorDiag2(sigma_alpha_beta,Rho)

Array([[[0.00630874],
        [0.00113279]],

       [[0.03548279],
        [0.19761124]]], dtype=float32)

In [10]:
m = {}
m['sigma']= tfd.Sample(tfd.Exponential(1, name = 'prior1'), sample_shape = 1)
m['sigma_alpha_beta']= tfd.Sample(tfd.Exponential(1, name = 'prior2'), sample_shape = 2)
m['alpha']= tfd.Sample(tfd.Normal(5,2, name = 'prior3'), sample_shape = 1)
m['beta']= tfd.Sample(tfd.Normal(-1,0.5, name = 'prior4'), sample_shape = 1)
m['Rho']= tfd.Sample(tfd.LKJ(int(2),int(2), name = 'prior5'), sample_shape = ())
m['a_cafe_b_cafe']= lambda sigma_alpha_beta, alpha, beta, Rho: tfd.Sample(
    tfd.MultivariateNormalTriL(jnp.concat([alpha,beta],axis=-1), jax_LinearOperatorDiag(sigma_alpha_beta,Rho)), sample_shape = 20)
m['y'] = lambda sigma, a_cafe_b_cafe : tfd.Independent(
    tfd.Normal( 
        jnp.squeeze(jnp.take(jnp.take(a_cafe_b_cafe.reshape(-1, 2), 0, axis=-1),
                             jnp.array(d.cafe.astype('float32').values, dtype=jnp.int32), axis = -1))+ 
        jnp.squeeze(jnp.take(jnp.take(a_cafe_b_cafe.reshape(-1, 2), 1, axis=-1),
                             jnp.array(d.cafe.astype('float32').values, dtype=jnp.int32), axis = -1))*
                             jnp.array(d.afternoon.astype('float32').values, dtype=jnp.int32),sigma, name ='main'), reinterpreted_batch_ndims=1)

M = tfd.JointDistributionNamedAutoBatched(m)
init_key, sample_key = random.split(random.PRNGKey(int(r.randint(0, 10000000))))
M.sample(seed=jnp.array(init_key))

/home/sosa/.local/lib/python3.10/site-packages/tensorflow_probability/python/internal/backend/jax/random_generators.py:283: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in zeros is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  minval = minval + np.zeros([1] * final_rank, dtype=dtype)
/home/sosa/.local/lib/python3.10/site-packages/tensorflow_probability/python/internal/backend/jax/random_generators.py:284: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'>  is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return jaxrand.randint(key=seed, shape=shape, minval=minval, maxval=maxv

{'sigma_alpha_beta': Array([2.7361476, 1.0094572], dtype=float32),
 'beta': Array([-0.43042916], dtype=float32),
 'alpha': Array([6.9925537], dtype=float32),
 'Rho': Array([[1.      , 0.924258],
        [0.924258, 1.      ]], dtype=float32),
 'a_cafe_b_cafe': Array([[ 6.275032  , -1.3218968 ],
        [ 5.9817414 , -1.2594076 ],
        [ 6.7409544 , -1.960676  ],
        [ 6.893266  , -0.42623556],
        [10.735725  , -0.16699097],
        [10.260581  ,  1.8899491 ],
        [ 5.1846514 , -1.9568622 ],
        [11.121883  , -0.51947665],
        [ 7.207047  , -0.05656007],
        [ 6.5271826 , -0.4595873 ],
        [ 7.793556  ,  0.6745506 ],
        [ 7.874182  , -2.12385   ],
        [11.077856  ,  1.0526431 ],
        [10.356081  ,  1.4226391 ],
        [ 8.668348  ,  1.3377683 ],
        [ 5.412852  , -1.7985299 ],
        [ 9.326059  , -1.355034  ],
        [ 5.669758  , -3.9023817 ],
        [ 2.1000443 , -1.7628951 ],
        [ 3.720598  , -2.9406874 ]], dtype=float32),
 'si

In [26]:
tfd.Sample(tfd.LKJ(int(2),int(2), name = 'prior5'), sample_shape = ())

<tfp.distributions.Sample 'Sampleprior5' batch_shape=[] event_shape=[2, 2] dtype=float32>

In [2]:
import math
import os

import arviz as az
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import Image, set_matplotlib_formats
from matplotlib.patches import Ellipse, transforms

import jax.numpy as jnp
from jax import random, vmap
from jax.scipy.special import expit

import numpy as onp
import numpyro as numpyro
import numpyro.distributions as dist
from numpyro.diagnostics import effective_sample_size, print_summary
from numpyro.infer import MCMC, NUTS, Predictive

az.style.use("arviz-darkgrid")
numpyro.set_platform("cpu")
numpyro.set_host_device_count(4)
a = 3.5  # average morning wait time
b = -1  # average difference afternoon wait time
sigma_a = 1  # std dev in intercepts
sigma_b = 0.5  # std dev in slopes
rho = -0.7  # correlation between intercepts and slopes
Mu = jnp.array([a, b])
cov_ab = sigma_a * sigma_b * rho
Sigma = jnp.array([[sigma_a**2, cov_ab], [cov_ab, sigma_b**2]])

sigmas = jnp.array([sigma_a, sigma_b])  # standard deviations
Rho = jnp.array([[1, rho], [rho, 1]])  # correlation matrix

# now matrix multiply to get covariance matrix
Sigma = jnp.diag(sigmas) @ Rho @ jnp.diag(sigmas)
N_cafes = 20
seed = random.PRNGKey(5)  # used to replicate example
vary_effects = dist.MultivariateNormal(Mu, Sigma).sample(seed, (N_cafes,))
a_cafe = vary_effects[:, 0]
b_cafe = vary_effects[:, 1]
seed = random.PRNGKey(22)
N_visits = 10
afternoon = jnp.tile(jnp.arange(2), N_visits * N_cafes // 2)
cafe_id = jnp.repeat(jnp.arange(N_cafes), N_visits)
mu = a_cafe[cafe_id] + b_cafe[cafe_id] * afternoon
sigma = 0.5  # std dev within cafes
wait = dist.Normal(mu, sigma).sample(seed)
d = pd.DataFrame(dict(cafe=cafe_id, afternoon=afternoon, wait=wait))

In [7]:
import time as tm
def model(cafe, afternoon, wait):
    a = numpyro.sample("a", dist.Normal(5, 2))
    b = numpyro.sample("b", dist.Normal(-1, 0.5))
    sigma_cafe = numpyro.sample("sigma_cafe", dist.Exponential(1).expand([2]))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    Rho = numpyro.sample("Rho", dist.LKJ(2, 2))
    cov = jnp.outer(sigma_cafe, sigma_cafe) * Rho
    a_cafe_b_cafe = numpyro.sample(
        "a_cafe,b_cafe", dist.MultivariateNormal(jnp.stack([a, b]), cov).expand([20])
    )
    a_cafe, b_cafe = a_cafe_b_cafe[:, 0], a_cafe_b_cafe[:, 1]
    mu = a_cafe[cafe] + b_cafe[cafe] * afternoon
    numpyro.sample("wait", dist.Normal(mu, sigma), obs=wait)

start = tm.time()    
m14_1 = MCMC(NUTS(model), num_warmup=500, num_samples=2000, num_chains=4)
m14_1.run(random.PRNGKey(0), jnp.array(d.cafe.values), jnp.array(d.afternoon.values), jnp.array(d.wait.values))
end = tm.time()    
print(f"Pystan took: {end - start:.4f} seconds")

Compiling.. :   0%|          | 0/2500 [00:00<?, ?it/s]







Running chain 0:  10%|█         | 250/2500 [00:02<00:00, 2387.99it/s]


Running chain 0:  25%|██▌       | 625/2500 [00:03<00:00, 3090.14it/s]


Running chain 0:  45%|████▌     | 1125/2500 [00:03<00:00, 3491.04it/s]


Running chain 0:  65%|██████▌   | 1625/2500 [00:03<00:00, 3675.82it/s]





Running chain 3: 100%|██████████| 2500/2500 [00:03<00:00, 706.43it/s] 


Pystan took: 3.8120 seconds


In [6]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()
stan_code = """ 
data{
    vector[200] wait;
    array[200] int afternoon;
    array[200] int cafe;
}
parameters{
    vector[20] b_cafe;
    vector[20] a_cafe;
    real a;
    real b;
    vector<lower=0>[2] sigma_cafe;
    real<lower=0> sigma;
    corr_matrix[2] Rho;
}
model{
    vector[200] mu;
    Rho ~ lkj_corr( 2 );
    sigma ~ exponential( 1 );
    sigma_cafe ~ exponential( 1 );
    b ~ normal( -1 , 0.5 );    
    a ~ normal( 5 , 2 );
    {
        array[20] vector[2] YY;
        vector[2] MU;
        MU = [ a , b ]';
        for ( j in 1:20 ) YY[j] = [ a_cafe[j] , b_cafe[j] ]';
        YY ~ multi_normal( MU , quad_form_diag(Rho , sigma_cafe) );
    }
    for ( i in 1:200 ) {
        mu[i] = a_cafe[cafe[i]] + b_cafe[cafe[i]] * afternoon[i];        
    }
    
    wait ~ normal( mu , sigma );

}
"""
data = {
    'wait' : d['wait'].values.astype(float),
    'afternoon' : d['afternoon'].values.astype(int),
    'cafe' : d['cafe'].values.astype(int)+1,
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_ybzqyjsi/model_y5f2lzzr.stan', line 18, column 4: It
    is suggested to reparameterize your model to replace lkj_corr with
    lkj_corr_cholesky, the Cholesky factor variant. lkj_corr tends to run
    slower, consume more memory, and has higher risk of numerical errors.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:


Pystan took: 3.0976 seconds


### 13.2. Output comparaison

In [7]:
pd.DataFrame(
    {
        "tfp": m14_1.summary(round_to='none')['mean'],
        "pystan": [df['sigma_cafe.1'].mean(),
                   df['sigma_cafe.2'].mean(),
                   df['b'].mean(),df['a'].mean(),
                   df['Rho.1.1'].mean(),df['Rho.2.1'].mean(),
                   df['Rho.1.2'].mean(),df['Rho.2.2'].mean(),
                   df['a_cafe.1'].mean(), df['b_cafe.1'].mean(),
                   df['a_cafe.2'].mean(), df['b_cafe.2'].mean(),
                   df['a_cafe.3'].mean(), df['b_cafe.3'].mean(),
                   df['a_cafe.4'].mean(), df['b_cafe.4'].mean(),
                   df['a_cafe.5'].mean(), df['b_cafe.5'].mean(),
                   df['a_cafe.6'].mean(), df['b_cafe.6'].mean(),
                   df['a_cafe.7'].mean(), df['b_cafe.7'].mean(),
                   df['a_cafe.8'].mean(), df['b_cafe.8'].mean(),
                   df['a_cafe.9'].mean(), df['b_cafe.9'].mean(),
                   df['a_cafe.10'].mean(), df['b_cafe.10'].mean(),
                   df['a_cafe.11'].mean(), df['b_cafe.11'].mean(),
                   df['a_cafe.12'].mean(), df['b_cafe.12'].mean(),
                   df['a_cafe.13'].mean(), df['b_cafe.13'].mean(),
                   df['a_cafe.14'].mean(), df['b_cafe.14'].mean(),
                   df['a_cafe.15'].mean(), df['b_cafe.15'].mean(),
                   df['a_cafe.16'].mean(), df['b_cafe.16'].mean(),
                   df['a_cafe.17'].mean(), df['b_cafe.17'].mean(),
                   df['a_cafe.18'].mean(), df['b_cafe.18'].mean(),
                   df['a_cafe.19'].mean(), df['b_cafe.19'].mean(),
                   df['a_cafe.20'].mean(), df['b_cafe.20'].mean(),
                   df['sigma'].mean()]
    })

,tfp,pystan
sigma_alpha_beta[0],0.069269,1.008199
sigma_alpha_beta[1],0.093032,0.636203
beta[0],-1.046420,-1.046213
alpha[0],3.681367,3.683057
"Rho[0, 0]",19.077908,1.000000
"Rho[0, 1]",7.663390,-0.638785
"Rho[1, 0]",-8.993556,-0.638785
"Rho[1, 1]",17.729902,1.000000
"a_cafe_b_cafe[0, 0]",3.022518,3.029072
"a_cafe_b_cafe[0, 1]",-0.955991,-0.975012


## 14. Gaussian process (WIP)

In [6]:
from src.main_jax import*
tfb = tfp.bijectors
# from: https://www.tensorflow.org/probability/examples/Gaussian_Process_Regression_In_TFP?hl=fr
def sinusoid(x):
  return np.sin(3 * np.pi * x[..., 0])

def generate_1d_data(num_training_points, observation_noise_variance):
  """Generate noisy sinusoidal observations at a random set of points.

  Returns:
     observation_index_points, observations
  """
  index_points_ = np.random.uniform(-1., 1., (num_training_points, 1))
  index_points_ = index_points_.astype(np.float64)
  # y = f(x) + noise
  observations_ = (sinusoid(index_points_) +
                   np.random.normal(loc=0,
                                    scale=np.sqrt(observation_noise_variance),
                                    size=(num_training_points)))
  return index_points_, observations_

# Generate training data with a known noise level (we'll later try to recover
# this value from the data).
NUM_TRAINING_POINTS = 100
observation_index_points_, observations_ = generate_1d_data(
    num_training_points=NUM_TRAINING_POINTS,
    observation_noise_variance=.1)

In [7]:
d = pd.DataFrame({'observation_index_points_' : tf.reshape(observation_index_points_, -1)})
tfk = tfp.math.psd_kernels
formula = dict(main = 'observations ~ GaussianProcess(kernel=tfp.math.psd_kernels.ExponentiatedQuadratic(amplitude, length_scale),index_points=observation_index_points_,observation_noise_variance=observation_noise_variance)',
               prior1 = 'observation_noise_variance ~ LogNormal(0,1)',
               prior2 = 'length_scale~LogNormal(0,1)',
               prior3 = 'amplitude~LogNormal(0,1)', 
               )
self = model(formula, d)
self.sample()

jax.local_device_count  32


IndexError: tuple index out of range

In [98]:
self.prior_dict

{'observation_noise_variance': "tfd.Sample(tfd.LogNormal(0,1, name = 'prior1'), sample_shape = 1)",
 'length_scale': "tfd.Sample(tfd.LogNormal(0,1, name = 'prior2'), sample_shape = 1)",
 'amplitude': "tfd.Sample(tfd.LogNormal(0,1, name = 'prior3'), sample_shape = 1)"}

In [91]:
self.main_dict

{'observations': "lambda observation_noise_variance, length_scale, amplitude : tfd.Independent(tfd.GaussianProcess(kernel=tfp.substrates.jax.math.psd_kernels.ExponentiatedQuadratic(amplitude,length_scale),index_points= df.observation_index_points_.astype('float32').values,observation_noise_variance=observation_noise_variance, name ='main'), reinterpreted_batch_ndims=1)"}

In [ ]:
from src.main import *
tfk = tfp.math.psd_kernels
m = {}
# GP prios 
m['amplitude'] = tfd.Sample(tfd.LogNormal(loc=0., scale=np.float64(1.)), sample_shape = ())
m['length_scale'] =  tfd.Sample(tfd.LogNormal(loc=0., scale=np.float64(1.)), sample_shape = ())
m['observation_noise_variance'] =  tfd.Sample(tfd.LogNormal(loc=0., scale=np.float64(1.)), sample_shape =())
m['observations'] = lambda amplitude, length_scale, observation_noise_variance: tfd.Independent(
      tfd.GaussianProcess(
            kernel=tfk.ExponentiatedQuadratic(amplitude, length_scale),
            index_points=observation_index_points_,
            observation_noise_variance=observation_noise_variance), reinterpreted_batch_ndims = 0)
M = tfd.JointDistributionNamed(m)
M.sample()

In [115]:
from src.main import*
m = {}
m['amplitude'] = tfd.Sample(tfd.LogNormal(0,1, name = 'prior3'), sample_shape = 1)
m['length_scale'] = tfd.Sample(tfd.LogNormal(0,1, name = 'prior2'), sample_shape = 1)
m['observation_noise_variance'] = tfd.Sample(tfd.LogNormal(0,1, name = 'prior1'), sample_shape = 1)
m['observations'] = lambda  observation_noise_variance, length_scale, amplitude: tfd.Independent(
    tfd.GaussianProcess(
        kernel=tfp.math.psd_kernels.ExponentiatedQuadratic(amplitude,length_scale),
        index_points= d.observation_index_points_.astype('float32').values,
        observation_noise_variance=observation_noise_variance, name ='main'), reinterpreted_batch_ndims=1)
m = tfd.JointDistributionNamedAutoBatched(m)
m

<tfp.distributions.JointDistributionNamedAutoBatched 'JointDistributionNamedAutoBatched' batch_shape=[] event_shape={amplitude: [1], length_scale: [1], observation_noise_variance: [1], observations: [1, 100]} dtype={amplitude: float32, length_scale: float32, observation_noise_variance: float32, observations: float32}>

In [87]:
m5_9.main_dict

{'K': "lambda sigma, alpha : tfd.Independent(tfd.Normal( jnp.squeeze(jnp.take(alpha,jnp.array(df.index_clade.astype('float32').values, dtype=jnp.int32), axis = -1)),sigma, name ='main'), reinterpreted_batch_ndims=1)"}

<font color='red'>**TODO**</font>: need to arrays 1:N to n:1